In [24]:
# Import findspark and initialize.
!pip install findspark 
import findspark
findspark.init()

In [25]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://"+SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


24/04/15 13:27:13 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


In [27]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [28]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
average_price_query = """
SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY Year
"""
average_price_result = spark.sql(average_price_query)
average_price_result.show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [29]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
average_price_built_query = """
SELECT date_built AS Year_Built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
average_price_built_result = spark.sql(average_price_built_query)
average_price_built_result.show()

+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [30]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
average_price_specific_query = """
SELECT date_built AS Year_Built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
WHERE bedrooms = 3 
  AND bathrooms = 3 
  AND floors = 2 
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
average_price_specific_result = spark.sql(average_price_specific_query)
average_price_specific_result.show()

+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [31]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
average_price_view_query = """
SELECT view, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
average_price_view_result = spark.sql(average_price_view_query)
average_price_view_result.show()




average_price_view_result = spark.sql(average_price_view_query)
average_price_view_result.show()
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|  

In [32]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
spark.sql("CACHE TABLE home_sales")
start_time_cached = time.time()
average_price_view_result_cached = spark.sql(average_price_view_query)
average_price_view_result_cached.show()
print("--- %s seconds ---" % (time.time() - start_time_cached))



24/04/15 13:27:17 WARN CacheManager: Asked to cache already cached data.


+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.3859391212463379 seconds ---


In [35]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
partitioned_parquet_path = "path_to_save_partitioned_parquet_data"
df.write.partitionBy("date_built").parquet(partitioned_parquet_path)


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/khalidnaji/Desktop/Mod22/path_to_save_partitioned_parquet_data already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
# 11. Read the formatted parquet data.
parquet_path = "/Users/khalidnaji/Desktop/Mod22/path_to_formatted_parquet_data"
df_parquet = spark.read.parquet(parquet_path)
df_parquet.show()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/khalidnaji/Desktop/Mod22/path_to_formatted_parquet_data.

In [ ]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("parquet_table")

NameError: name 'df_parquet' is not defined

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.


start_time = time.time()
average_price_view_result_parquet = spark.sql(average_price_view_query)
average_price_view_result_parquet.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.7652318477630615 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached


cached_tables = spark.catalog.listTables()
home_sales_cached = any(table.name == "home_sales" for table in cached_tables)
if home_sales_cached:
    print("home_sales is still cached.")
else:
    print("home_sales is not cached anymore.")

home_sales is still cached.
